In [2]:
import os

os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
import cudf
import glob
import gc

import pandas as pd

from tqdm import tqdm

In [4]:
def calc_recall(sub, ty='clicks', file='./data/xgb_train_y.parquet'):
    sub = sub.sort_values(['session', 'score'], ascending=[True, False])
    sub['dummy'] = 1
    sub['rank'] = sub.groupby(['session']).dummy.cumsum()
    sub.drop(['dummy'], axis=1, inplace=True)
    sub = sub[sub['rank']<21]
    sub = sub[['session', 'cand']]
    sub.columns = ['session', 'aid']
    test_labels = cudf.read_parquet(file)
    test_labels = test_labels[test_labels['type']==ty]
    test_labels = test_labels.merge(
        test_labels.groupby(['session', 'type']).count().reset_index().rename(columns={'aid': 'no_gt'}),
        how='left',
        on=['session', 'type']
    )
    sub['target'] = 1
    sub.columns = ['session', 'aid', 'target']
    test_labels = test_labels.merge(
        sub[['session', 'aid', 'target']].drop_duplicates(['session', 'aid']),
        how='left',
        on=['session', 'aid']

    )
    test_labels['target'] = test_labels['target'].fillna(0)
    test_labels = test_labels[test_labels['session'].isin(sub['session'])]
    test_labels = test_labels.groupby(['session', 'aid']).agg({'no_gt': 'min', 'target': 'sum'}).reset_index()
    recall = test_labels.target.sum()/test_labels.groupby(['session']).no_gt.min().clip(0,20).sum()
    print(recall)
    return(recall, test_labels.target.sum(), test_labels.groupby(['session']).no_gt.min().clip(0,20).sum())

In [5]:
!ls ./data/split/chunks_c/

ls: cannot access './data/split/chunks_c/': No such file or directory


In [7]:
import xgboost as xgb

xgb_parms = { 
    'max_depth':8, 
    'learning_rate':0.1, 
    'subsample':0.8,
    'colsample_bytree':0.3, 
    'eval_metric':'logloss',
    'objective':'binary:logistic',
    'tree_method':'gpu_hist' 
}

bl_sub = True
bl_pos = True
outputfolder = '24_XGB_Rerun_RedruceCandidate_DifferentWeights_Folds_ChrisCo_SameDay_v9'
no_seeds = 10
no_trees = 345

In [8]:
ty = 'orders'
labels = cudf.read_parquet('./data/xgb_train_y.parquet')
labels.columns = ['session', 'cand', 'type']
labels['target'] = 1

In [9]:
files = sorted(glob.glob('./data/split/chunks/' + ty + '/chunk*.parquet'))
files

['./data/split/chunks/orders/chunk_0.parquet',
 './data/split/chunks/orders/chunk_1.parquet']

In [10]:
[y for x in [
    sorted(glob.glob('./data_folds/fold_' + str(igfold2) + '/split/chunks/' + ty + '/chunk*.parquet'))
    for igfold2 in range(0,5)
] for y in x ]

['./data_folds/fold_0/split/chunks/orders/chunk_0.parquet',
 './data_folds/fold_0/split/chunks/orders/chunk_1.parquet',
 './data_folds/fold_1/split/chunks/orders/chunk_0.parquet',
 './data_folds/fold_1/split/chunks/orders/chunk_1.parquet',
 './data_folds/fold_2/split/chunks/orders/chunk_0.parquet',
 './data_folds/fold_2/split/chunks/orders/chunk_1.parquet',
 './data_folds/fold_3/split/chunks/orders/chunk_0.parquet',
 './data_folds/fold_3/split/chunks/orders/chunk_1.parquet',
 './data_folds/fold_4/split/chunks/orders/chunk_0.parquet',
 './data_folds/fold_4/split/chunks/orders/chunk_1.parquet']

In [11]:
session_filter = labels[(labels['type']=='orders')]['session'].drop_duplicates().to_pandas().tolist()

In [12]:
ingore_cols = []

In [ ]:
%%time

models = []
hist_recall = []
total_no_hit = 0
total_no_gt = 0
for igfold in range(1):
    files = ['x']
    print(files)
    for ifile, file in enumerate(files):
        if bl_sub:
            if ty == 'clicks':
                test_files = []
                train_files = [files[x] for x in [(ifile+i+1)%len(files) for i in range(len(files))]]
            elif ty == 'carts':
                test_files = []
                train_files = [files[x] for x in [(ifile+i+1)%len(files) for i in range(len(files))]]
            elif ty == 'orders':
                test_files = []
                train_files = [files[x] for x in [(ifile+i+1)%len(files) for i in range(len(files))]]
            else:
                assert 0!=0
            
            train_files = sorted([
                y for x in [
                    sorted(glob.glob('./data_folds/fold_' + str(igfold2) + '/split/chunks_c_2/' + ty + '/chunk*.parquet'))
                    for igfold2 in range(0,5)
                ] for y in x
            ])
            
#            step=5
#            train_files_list = []
#            for i in range(0, len(train_files), step):
#                x = i
#                train_files_list.append(train_files[x:x+step])
#            train_files = train_files_list[ifile]
#            print(train_files)
            
            if ty == 'clicks':
                df_train = pd.concat([
                    pd.read_parquet(x) for x in train_files
                ])
            elif ty == 'carts':
                df_train = pd.concat([
                    pd.read_parquet(x) for x in train_files
                ])
            elif ty == 'orders':
                df_train = pd.concat([
                    pd.read_parquet(x) for x in train_files
                ])
            #df_train = df_train[~(df_train['session'].isin(session_filter))]

        print(train_files)
        print(test_files)

        if bl_pos:
            df_train = df_train[df_train['session'].isin(
                df_train[df_train['target']==1]['session'].drop_duplicates().values
            )]

        train_cols = [x for x in df_train.columns if x not in [
            'session', 'cand', 'target', 'target_clicks', 'target_carts', 'target_orders'
        ] + ingore_cols]

        print('Recall Train: ' + str(df_train[df_train['target']==1].shape[0]/labels[
        (labels['session'].isin(df_train['session'].drop_duplicates()))&(labels['type']==ty)
    ].shape[0]))
        if not bl_sub:
            print('Recall Test: ' + str(df_test[df_test['target']==1].shape[0]/labels[
            (labels['session'].isin(df_test['session'].drop_duplicates()))&(labels['type']==ty)
        ].shape[0]))

        dtrain = xgb.DMatrix(data=df_train[train_cols].values,label=df_train['target'].values)
        if not bl_sub:
            dtest =  xgb.DMatrix(data=df_test[train_cols].values, label=df_test['target'].values)
            df_test['score'] = 0.0

        del df_train
        gc.collect()

        for iseed in range(no_seeds):
            print('iseed: ' + str(iseed))
            xgb_parms['seed'] = iseed
            if bl_sub:
                model = xgb.train(
                    xgb_parms, 
                    dtrain=dtrain,
                    evals=[(dtrain,'train')],
                    num_boost_round=no_trees,
                    verbose_eval=10
                )
            else:
                model = xgb.train(
                    xgb_parms, 
                    dtrain=dtrain,
                    evals=[(dtest,'test')],
                    num_boost_round=no_trees,
                    verbose_eval=10,
                    early_stopping_rounds=50
                )
                df_test['score'] += model.predict(dtest)
            models.append(model)
        del dtrain
        
        if bl_sub:
            for igfold2 in range(5):
                for isubfile, sub_file in enumerate(glob.glob('./data_folds/fold_' + str(igfold2) + '/split/sub_c_2/*.parquet')):
                    print('isubfile: ' + str(isubfile))
                    df_sub = cudf.read_parquet(sub_file).fillna(-999)
                    df_sub['score'] = 0.0
                    dsub = xgb.DMatrix(data=df_sub[train_cols].values)
                    for iseed in range(no_seeds):
                        df_sub['score_' + str(iseed)] = models[iseed].predict(dsub)
                    iseed = 0
                    os.system('mkdir -p ../00_submissions/' + outputfolder)
                    df_sub[['session', 'cand'] + ['score_' + str(ij) for ij in range(no_seeds)]].to_parquet(
                        '../00_submissions/' + outputfolder + '/sub_' + str(igfold2) + '_' + ty + '_' + str(ifile) + '_' + str(isubfile) + '.parquet'
                    )
                    del df_sub
                    gc.collect()
            del model
            gc.collect()

        gc.collect()